In [ ]:
# @title Instalación de Dependencias
!pip install mysql-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.6/31.6 MB 45.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.8/409.8 kB 48.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.21.12 which is incompatible.


In [ ]:
# @title Import Libraries
import pandas as pd
import json
import os
import mysql.connector

Se han elegido dos tablas para nuestra base de datos. Una con toda la información de los viajes y relacionados con estos y luego otra con los datos sobre las estaciones en base a su disponibilidad (Cantidad disponible, Maxima ocupación y demás)

# Datos de Viajes

## Inicio de Queries  
Aqui declaramos las queries para la creación de la tabla como la inserción de datos.

In [ ]:
# @markdown Los datos que nos interesan son los siguientes: Fecha, Duración del viaje, Fecha Desbloqueo, Fecha Bloqueo,
# @markdown ID Lugar Desbloqueo, Nombre Lugar de Desbloqueo, ID Lugar Bloqueo, Nombre Lugar de Bloqueo y Coordenadas de cada zona
query_create_travels = "CREATE TABLE travels (dates DATE, trip_minutes FLOAT, unlock_date TIMESTAMP," \
        "lock_date TIMESTAMP, station_unlock VARCHAR(255), unlock_station_name VARCHAR(255), " \
        "station_lock VARCHAR(255), lock_station_name VARCHAR(255) , unlock_latitude FLOAT, unlock_longitude FLOAT, " \
        "lock_latitude FLOAT, lock_longitude FLOAT)"

query_insert_travels = "INSERT INTO travels (dates, trip_minutes, unlock_date," \
        "lock_date, station_unlock, unlock_station_name , " \
        "station_lock , lock_station_name , unlock_latitude, unlock_longitude, " \
        "lock_latitude, lock_longitude) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

In [ ]:
# @markdown Iniciamos la conexion con la base de datos
pd.set_option('display.max_colwidth', 255)

host = "" # @param {type: "string"}
user = "" # @param {type: "string"}
password = "" # @param {type: "string"}
database = "" # @param {type: "string"}

mydb = mysql.connector.connect(
  host=host,
  user=user,
  password=password,
  database=database
)

mycursor = mydb.cursor()

## Preparación Datos de Viajes  
Antes de poder meter los datos en nuestra base de datos tenemos que analizar estos y ajustarlos a nuestras necesidades. No hay que hacer mucha limpieza de datos realmente, solo tener cuidado a la hora de leer los archivos ya que hay diferencias entre ficheros.  
Con el siguiente codigo nos aseguramos que todos los archivos queden con los mismos datos y listos para insertar en la base de datos.


In [ ]:
# @title Limpieza de Datos e inserción en la base de datos
# @markdown Para mayor eficiencia se leen los archivos csv de dentro de una carpeta y se iteran haciendo las mismas operaciones en ellos.
# @markdown Por último se insertan los datos del csv en la tabla correspondiente en la base de datos.
path_folder_files = "Trips" # @param {type: "string"}
for file in os.listdir(path_folder_files):
    df = pd.read_csv(f"{path_folder_files}\\{file}", sep=";")
    df = df.drop(columns=["address_unlock", "address_lock"])
    df = df.dropna().reset_index().drop(columns=["index"])
    unlock_lat = []
    unlock_lon = []
    for d in df["geolocation_unlock"]:
        d = json.loads(d.replace("'", "\""))
        unlock_lat.append(d["coordinates"][1])
        unlock_lon.append(d["coordinates"][0])

    lock_lat = []
    lock_lon = []
    for d in df["geolocation_lock"]:
        d = json.loads(d.replace("'", "\""))
        lock_lat.append(d["coordinates"][1])
        lock_lon.append(d["coordinates"][0])


    df = df.drop(columns=["idBike", "geolocation_unlock", "geolocation_lock", "unlocktype", "locktype", "fleet",
                          "dock_unlock", "dock_lock"])
    try:
        df = df.drop(columns=["idTrip"])
    except:
        pass

    df["unlock_latitude"] = unlock_lat
    df["unlock_longitude"] = unlock_lon

    df["lock_latitude"] = lock_lat
    df["lock_longitude"] = lock_lon

    df["unlock_date"] = df["unlock_date"].apply(lambda x: x.replace("T", " "))
    df["lock_date"] = df["lock_date"].apply(lambda x: x.replace("T", " "))

    df.to_csv("good_data.csv", index=False)

    df = pd.read_csv("good_data.csv")

    tuple_list = [tuple(x) for x in df.to_records(index=False)]

    mycursor.executemany(query_insert_travels, tuple_list)
    mydb.commit()
    print(f"File {file} inserted into the DB")

Como se ha comentado no hay muchos detalles en la limpieza, si hay algo a destacar es formatear las fechas para que queden a corde a las variables de la base de datos y poco mas. Se han quitado algunas columnas que no aportaban información relevante o que no diese ya otra.

# Datos de Estaciones

## Inicio de Queries  
Aqui declaramos las queries para la creación de la tabla como la inserción de datos.

In [ ]:
query_create_stations = "CREATE TABLE stations (max_occupation INT, free_bases INT, longitude FLOAT, no_available INT," \
        " address VARCHAR(255), latitude FLOAT, bikes_available INT, " \
        "id INT, date DATE, hours TIME)"

query_insert_statation = "INSERT INTO stations (max_occupation, free_bases, longitude, no_available," \
            " address, latitude, bikes_available, " \
            "id, date, hours) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

## Preparación Datos sobre las Estaciones
Lo mismo que se ha realizado antes, se procede a la limpieza de los datos para posteriormente añadirlos a la base de dato. En este caso se tratan de archivos JSON y se leen desde una carpeta al igual que antes.


In [ ]:
path_folder_files = "" # @param {type: "string"}
for file in os.listdir(path_folder_files):
    df = pd.read_json(f"{path_folder_files}\\{file}", lines=True)
    df["date"] = df["_id"].apply(lambda x: x.split("T")[0])
    df["hours"] = df["_id"].apply(lambda x: x.split("T")[1].split(".")[0])
    df = df.drop(columns=["_id"])
    true_df = pd.DataFrame()
    for i in range(len(df)):
        row = df["stations"].iloc[i]
        aux = pd.DataFrame(row)
        aux["date"] = df["date"][i]
        aux["hours"] = df["hours"][i]
        true_df = pd.concat([true_df, aux])

    true_df = true_df.drop(columns=["activate", "reservations_count", "light", "number", "name"])
    for i in true_df.columns:
        try:
            true_df[i] = true_df[i].astype(float)
        except:
            pass
    true_df.to_csv("stations_info.csv", index=False)

    tuple_list = [tuple(x) for x in true_df.to_records(index=False)]

    mycursor.executemany(query_insert_statation, tuple_list)
    mydb.commit()

En esta parte es verdad que hay más dificultad ya que se trata de un JSON pero en terminos de limpieza sigue sin haber muchos. Simplemente colocar los datos de manera que coincidad los de todos los archivos

Y con esto ya tendriamos la base de datos montada con las dos tablas de datos que queremos. Este notebook ha sido ejecutado en local, se recomienda hacer lo mismo montando la base de datos en una maquina propia.